In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import requests
from bs4 import BeautifulSoup as soup

In [2]:
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set()

In [6]:
#read csv
path = 'nfl_games_2019.csv'
df = pd.read_csv(path)
df
df.head()
# df.describe()

,game_id,home_team,away_team,week,season,home_score,away_score
0,2019090500,CHI,GB,1,2019,3,10
1,2019090800,CAR,LA,1,2019,27,30
2,2019090806,PHI,WAS,1,2019,32,27
3,2019090805,NYJ,BUF,1,2019,16,17
4,2019090804,MIN,ATL,1,2019,28,12


In [ ]:
pd.con

#create a new col called net_score = home - away
#initialize ratings for each team at the start of the season
##use data from nfl
- use 2 dfs: one for game data and another for weekly elo rating
- elo rating DF: 32 rows, each for a team and elo ratings for each week as a column. This setup makes it easy to sort by ELO ratings by week.
- game DF: add the following columns: home team's initial ELO, away team's initial ELO, expected home score, observe game score, home team's ELO change, updated home team'ELO, updated away team's ELO
    - ok.home team's initial ELO: use boolean indexing on ELO-DF and assign resulting series to game-df
    - OK same for away team's inital ELO
    - OK expected home score: apply func on both teams initial ELO
    - OK observed score: from the perspective of the home team: 1 if home team win, 0.5 if a tie, 0 is lose
    - OK homeTeam ELO change: apply func on expected home score and observed score.
    - OK updated homeTeam ELO: current homeTeam's ELO + ELO change
    - OK updated awayTeam ELO: current awayTeam's ELO - ELO change
- OK add updated ELO ratings to team


In [230]:
def calc_expected_score(df, team_rating_column, opposing_team_rating_column):
    return 1/ (1 + 10**((df[opposing_team_rating_column] - df[team_rating_column])/400))

def calc_ELO_change(df, observed_score_column, expected_score_column, k_factor=20):
    return k_factor * (df[observed_score_column] - df[expected_score_column])

In [231]:
elo_df

,week_0,week_1
team,,
ARI,0,0.287744
ATL,1,-8.482381
BAL,2,12.460192
BUF,3,13.575011
CAR,4,-5.654750
CHI,5,-4.827323
CIN,6,-3.396306
CLE,7,-2.338972
DAL,8,18.402734


,game_id,home_team,away_team,week,season,home_score,away_score
0,2019090500,CHI,GB,1,2019,3,10
1,2019090800,CAR,LA,1,2019,27,30
2,2019090806,PHI,WAS,1,2019,32,27
3,2019090805,NYJ,BUF,1,2019,16,17
4,2019090804,MIN,ATL,1,2019,28,12
5,2019090803,MIA,BAL,1,2019,10,59
6,2019090802,JAX,KC,1,2019,26,40
7,2019090801,CLE,TEN,1,2019,13,43
8,2019090807,LAC,IND,1,2019,30,24
9,2019090808,SEA,CIN,1,2019,21,20


In [165]:
elo_df

,week_0,week_1
team,,
ARI,0,ARI
ATL,1,ATL
BAL,2,BAL
BUF,3,BUF
CAR,4,CAR
CHI,5,CHI
CIN,6,CIN
CLE,7,CLE
DAL,8,DAL


In [221]:
#create df for chosen week copied from the main data
weekNum = 1
week_games = df[df.week==weekNum].copy()
new_week_col = 'week_' + str(weekNum)

#create the ELO rating dataframe where each row is a team and each column is ELO ratings of that week
elo_df = pd.DataFrame(pd.Series(team_names), columns=['team'])
init_elo = np.arange(32)
elo_df['week_0'] = init_elo
#set up new week's elo to be the name of teams.
elo_df[new_week_col] = elo_df['team']
elo_df = elo_df.set_index('team', drop=True)

In [222]:
elo_df, week_games

(      week_0 week_1
 team               
 ARI        0    ARI
 ATL        1    ATL
 BAL        2    BAL
 BUF        3    BUF
 CAR        4    CAR
 CHI        5    CHI
 CIN        6    CIN
 CLE        7    CLE
 DAL        8    DAL
 DEN        9    DEN
 DET       10    DET
 GB        11     GB
 HOU       12    HOU
 IND       13    IND
 JAX       14    JAX
 KC        15     KC
 LA        16     LA
 LAC       17    LAC
 MIA       18    MIA
 MIN       19    MIN
 NE        20     NE
 NO        21     NO
 NYG       22    NYG
 NYJ       23    NYJ
 OAK       24    OAK
 PHI       25    PHI
 PIT       26    PIT
 SEA       27    SEA
 SF        28     SF
 TB        29     TB
 TEN       30    TEN
 WAS       31    WAS,
        game_id home_team away_team  week  season  home_score  away_score
 0   2019090500       CHI        GB     1    2019           3          10
 1   2019090800       CAR        LA     1    2019          27          30
 2   2019090806       PHI       WAS     1    2019          32  

In [ ]:
### what even is this shit
home_elos = w1[['home_team', 'updated_home_team_ELO']]
home_elos = home_elos.set_index('home_team',drop=True)
elos = home_elos.to_dict()['updated_home_team_ELO']


In [180]:
type(B.set_index('Code')['Value'])

pandas.core.series.Series

In [177]:
A = pd.DataFrame(
    {'C1': ['a', 'd', 'a', 'b'],
     'C2': ['b', 'a', 'c', 'e'],
     'C3': ['c', 'e', '', 'e'],
     'C4': ['a', 'b', '', ''],
     'C5': ['', 'a', '', '']})
B = pd.DataFrame({'Code': ['a', 'b', 'c', 'd', 'e'],
                  'Value': ["'House'", "'Bike'", "'Lamp'", "'Window'", "'Car'"]})
print(A.replace(B.set_index('Code')['Value']))

         C1       C2      C3       C4       C5
0   'House'   'Bike'  'Lamp'  'House'         
1  'Window'  'House'   'Car'   'Bike'  'House'
2   'House'   'Lamp'                          
3    'Bike'    'Car'   'Car'                  


In [223]:
###
week_games['home_team_current_ELO'] = week_games[['home_team']].replace(elo_df['week_0'])
week_games['away_team_current_ELO'] = week_games[['away_team']].replace(elo_df['week_0'])
week_games

week_games['expected_home_score'] = calc_expected_score(week_games, 'home_team_current_ELO', 'away_team_current_ELO')
week_games


# create observed_score
week_games['observed_score'] = week_games.home_score - week_games.away_score
week_games['observed_score'][week_games['observed_score']>0] = 1
week_games['observed_score'][week_games['observed_score']==0] = 0.5
week_games['observed_score'][week_games['observed_score']<0] = 0
week_games



### create new cols homeTeam_ELO_change, updated_home_team_ELO, updated_away_team_ELO
week_games['homeTeam_ELO_change'] = calc_ELO_change(week_games, 'observed_score', 'expected_home_score')
# w1
week_games['updated_home_team_ELO'] = week_games.home_team_current_ELO + week_games.homeTeam_ELO_change
week_games['updated_away_team_ELO'] = week_games.away_team_current_ELO - week_games.homeTeam_ELO_change
week_games

###create a combined updated ELO rating df.
#### use this df to update elo-df

elos = week_games[['home_team', 'updated_home_team_ELO']]
new_col_names = ['team', 'elo']
elos.columns = new_col_names
temp = week_games[['away_team', 'updated_away_team_ELO']]
temp.columns = new_col_names
elos= elos.append(temp)
elos = elos.set_index('team', drop=True)
elos

## update the new ELO ratings for the current week
#name of week
new_week = 'week_1'
##add data
elo_df[new_week] = elo_df[[new_week]].replace(elos['elo'])

<ipython-input-223-7535f9ead820>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_games['observed_score'][week_games['observed_score']>0] = 1
<ipython-input-223-7535f9ead820>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_games['observed_score'][week_games['observed_score']==0] = 0.5


In [227]:
week_games

,game_id,home_team,away_team,week,season,home_score,away_score,home_team_current_ELO,away_team_current_ELO,expected_home_score,observed_score,homeTeam_ELO_change,updated_home_team_ELO,updated_away_team_ELO
0,2019090500,CHI,GB,1,2019,3,10,5,11,0.491366,0.0,-9.827323,-4.827323,20.827323
1,2019090800,CAR,LA,1,2019,27,30,4,16,0.482737,0.0,-9.654750,-5.654750,25.654750
2,2019090806,PHI,WAS,1,2019,32,27,25,31,0.491366,1.0,10.172677,35.172677,20.827323
3,2019090805,NYJ,BUF,1,2019,16,17,23,3,0.528751,0.0,-10.575011,12.424989,13.575011
4,2019090804,MIN,ATL,1,2019,28,12,19,1,0.525881,1.0,9.482381,28.482381,-8.482381
5,2019090803,MIA,BAL,1,2019,10,59,18,2,0.523010,0.0,-10.460192,7.539808,12.460192
6,2019090802,JAX,KC,1,2019,26,40,14,15,0.498561,0.0,-9.971218,4.028782,24.971218
7,2019090801,CLE,TEN,1,2019,13,43,7,30,0.466949,0.0,-9.338972,-2.338972,39.338972
8,2019090807,LAC,IND,1,2019,30,24,17,13,0.505756,1.0,9.884876,26.884876,3.115124
9,2019090808,SEA,CIN,1,2019,21,20,27,6,0.530185,1.0,9.396306,36.396306,-3.396306


In [ ]:
###create home_team_current_ELO column
#create new Series of home teams current ELO ratings
home_current_ELO = elo_df[elo_df.team.isin(w1.home_team)]
# type(home_current_ELO)
w1 = pd.merge(w1, home_current_ELO, left_on='home_team', right_on='team')
w1 = w1.rename(columns={"Week 0": "home_team_current_ELO"})
w1
###do the same for away team ELO
away_current_ELO = elo_df[elo_df.team.isin(w1.away_team)]
away_current_ELO = away_current_ELO.rename(columns={"team": "away_team"})
# type(home_current_ELO)
w1 = pd.merge(w1, away_current_ELO, on='away_team', validate="one_to_one")
w1 = w1.rename(columns={"Week 0": "away_team_current_ELO"})
w1

w1['expected_home_score'] = calc_expected_score(w1, 'home_team_current_ELO', 'away_team_current_ELO')
# w1['expected_away_score'] = 1 - w1['expected_home_score']
# w1.drop(['expected_away_score'])
# w1 = w1.drop(['expected_away_score'], axis=1)
w1

# create observed_score
w1['observed_score'][w1['observed_score']>0] = 1
w1['observed_score'][w1['observed_score']==0] = 0.5
w1['observed_score'][w1['observed_score']<0] = 0
w1


### create new cols homeTeam_ELO_change, updated_home_team_ELO, updated_away_team_ELO
w1['homeTeam_ELO_change'] = calc_ELO_change(w1, 'observed_score', 'expected_home_score')
# w1
w1['updated_home_team_ELO'] = w1.home_team_current_ELO + w1.homeTeam_ELO_change
w1['updated_away_team_ELO'] = w1.away_team_current_ELO - w1.homeTeam_ELO_change
w1

###create a combined updated ELO rating df.
#### use this df to update elo-df

elos = w1[['home_team', 'updated_home_team_ELO']]
new_col_names = ['team', 'elo']
elos.columns = new_col_names
temp = w1[['away_team', 'updated_away_team_ELO']]
temp.columns = new_col_names
elos= elos.append(temp)
elos = elos.set_index('team', drop=True)
elos

In [ ]:
new_week = 'week_1'
elo_df[new_week] = elo_df[teams]
elo_df[new_week] = elo_df.replace(elos)

In [143]:
#update ELO ratings in elo-df
new_week = 'week_1'
#Q: index doesn't have to match??
elo_df[new_week] = elo_df.replace(w1.set_index('home_team')['updated_home_team_ELO'])
elo_df[new_week] = elo_df.replace(w1.set_index('away_team')['updated_away_team_ELO'])

,team,Week 0,week_1
0,1.287744,1,1.287744
1,-7.482381,2,-7.482381
2,13.460192,3,13.460192
3,14.575011,4,14.575011
4,-4.654750,5,-4.654750
5,-3.827323,6,-3.827323
6,-2.396306,7,-2.396306
7,-1.338972,8,-1.338972
8,19.402734,9,19.402734
9,0.431467,10,0.431467


In [ ]:
#merge values
#create new column in ELO-df
#merge doesn't work!!!
elo_df = pd.merge(elo_df, w1[['home_team', 'updated_home_team_ELO']], left_on='team', right_on='home_team', how='outer')
elo_df= elo_df.rename(columns={"updated_home_team_ELO": "week_1"})
elo_df =  elo_df.drop(['home_team'], axis=1)
elo_df = pd.merge(elo_df, w1[['away_team', 'updated_away_team_ELO']], left_on='team', right_on='away_team', how='outer')

,game_id,home_team,away_team,week,season,home_score,away_score,team,home_team_current_ELO,away_team_current_ELO,expected_home_score
0,2019090500,CHI,GB,1,2019,3,10,CHI,6,12,0.491366
1,2019090800,CAR,LA,1,2019,27,30,CAR,5,17,0.482737
2,2019090806,PHI,WAS,1,2019,32,27,PHI,26,32,0.491366
3,2019090805,NYJ,BUF,1,2019,16,17,NYJ,24,4,0.528751
4,2019090804,MIN,ATL,1,2019,28,12,MIN,20,2,0.525881
5,2019090803,MIA,BAL,1,2019,10,59,MIA,19,3,0.523010
6,2019090802,JAX,KC,1,2019,26,40,JAX,15,16,0.498561
7,2019090801,CLE,TEN,1,2019,13,43,CLE,8,31,0.466949
8,2019090807,LAC,IND,1,2019,30,24,LAC,18,14,0.505756
9,2019090808,SEA,CIN,1,2019,21,20,SEA,28,7,0.530185


In [152]:
###create a combined updated ELO rating df.
#### use this df to update elo-df

elos = w1[['home_team', 'updated_home_team_ELO']]
new_col_names = ['team', 'elo']
elos.columns = new_col_names
temp = w1[['away_team', 'updated_away_team_ELO']]
temp.columns = new_col_names
elos= elos.append(temp)
elos = elos.set_index('team', drop=True)
elos

,elo
team,
CHI,-3.827323
CAR,-4.654750
PHI,36.172677
NYJ,13.424989
MIN,29.482381
MIA,8.539808
JAX,5.028782
CLE,-1.338972
LAC,27.884876


In [97]:
w1 = df[df.week==1]
w1 = w1.copy()
w1

elo_df['Week 0'] = np.arange(0, 3300 ,33)

,game_id,home_team,away_team,week,season,home_score,away_score,team,home_team_current_ELO
0,2019090500,CHI,GB,1,2019,3,10,CHI,6
1,2019090800,CAR,LA,1,2019,27,30,CAR,5
2,2019090806,PHI,WAS,1,2019,32,27,PHI,26
3,2019090805,NYJ,BUF,1,2019,16,17,NYJ,24
4,2019090804,MIN,ATL,1,2019,28,12,MIN,20
5,2019090803,MIA,BAL,1,2019,10,59,MIA,19
6,2019090802,JAX,KC,1,2019,26,40,JAX,15
7,2019090801,CLE,TEN,1,2019,13,43,CLE,8
8,2019090807,LAC,IND,1,2019,30,24,LAC,18
9,2019090808,SEA,CIN,1,2019,21,20,SEA,28


In [98]:


sum(w1.home_team_current_ELO) + sum(w1.away_team_current_ELO) - sum(np.arange(1, 33))



w1 = df[df.week==1]
w1
#select home team
type(w1.home_team)
#select away team
#combine to get all 32 teams
teams = pd.concat([w1.home_team, w1.away_team])
teams = teams.sort_values()
teams = teams.reset_index(drop=True)
teams
w0_rating = np.full_like(teams, 1500)
s1 = pd.Series(w0_rating)
w0_rating = pd.DataFrame(s1, columns=['Week 0'])
#below doesn't work, why?
# elo_df = pd.DataFrame([teams, w0_rating], columns=['team', 'week 0'])
elo_df = pd.DataFrame(teams, columns=['team'])
elo_df
elo_df['Week 0'] = s1
elo_df;

,game_id,home_team,away_team,week,season,home_score,away_score,team,home_team_current_ELO,away_team_current_ELO
0,2019090500,CHI,GB,1,2019,3,10,CHI,6,12
1,2019090800,CAR,LA,1,2019,27,30,CAR,5,17
2,2019090806,PHI,WAS,1,2019,32,27,PHI,26,32
3,2019090805,NYJ,BUF,1,2019,16,17,NYJ,24,4
4,2019090804,MIN,ATL,1,2019,28,12,MIN,20,2
5,2019090803,MIA,BAL,1,2019,10,59,MIA,19,3
6,2019090802,JAX,KC,1,2019,26,40,JAX,15,16
7,2019090801,CLE,TEN,1,2019,13,43,CLE,8,31
8,2019090807,LAC,IND,1,2019,30,24,LAC,18,14
9,2019090808,SEA,CIN,1,2019,21,20,SEA,28,7


In [156]:
team_names = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'MIA',
       'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS']

In [228]:
def calc_expected_score(team_rating, opposing_team_rating):
    return 1/ (1 + 10**((opposing_team_rating - team_rating)/400))

def calc_new_rating(team_rating, observed_score, expected_score, k_factor=20):
    return team_rating + k_factor * (observed_score - expected_score)

In [229]:
PIT = 0
NE = 10
#PIT's expected score
exp_score = calc_expected_score(PIT, NE)
calc_new_rating(PIT, 0.5, calc_expected_score(PIT, NE))
calc_new_rating(NE, 0.5, calc_expected_score(NE, PIT))

9.712256316680026

In [11]:
calc_expected_score(PIT, NE) + calc_expected_score(NE, PIT)

1.0

In [ ]:
# game df: game_ide, homeTeam, awayTeam, homescore, away score, observed_scoed(home persepctive), Expected_score, homeTeamRatingChange, 
# ELO rating by week df


In [ ]:
#create a new df: 32 rows for 32 teams
#each col = each week
#week 0 ELO initialized
#ea week: find matches played ea week. calc Expec score. calc observed score. update ranking

#find games played each week
#calc expected scores, assign to respective teams.
#add to dict
#


In [ ]:
# ELO ratings for ea week as a col:
# advantage: can sort_values to see which team is the best each week


In [157]:
pd.Series(team_names)

0     ARI
1     ATL
2     BAL
3     BUF
4     CAR
5     CHI
6     CIN
7     CLE
8     DAL
9     DEN
10    DET
11     GB
12    HOU
13    IND
14    JAX
15     KC
16     LA
17    LAC
18    MIA
19    MIN
20     NE
21     NO
22    NYG
23    NYJ
24    OAK
25    PHI
26    PIT
27    SEA
28     SF
29     TB
30    TEN
31    WAS
dtype: object